In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizontal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,61105.20,-0.17,-0.04,-0.13,-0.05,-0.00,-0.01,0.00
3,-0.17,11387.14,-0.02,0.03,0.01,0.01,0.00,-0.00
4,-0.04,-0.02,2046.07,-0.03,-0.00,0.01,-0.00,0.00
5,-0.13,0.03,-0.03,329.06,0.00,-0.00,0.00,-0.00
6,-0.05,0.01,-0.00,0.00,51.75,-0.00,0.00,0.00
7,-0.00,0.01,0.01,-0.00,-0.00,7.56,-0.00,-0.00
8,-0.01,0.00,-0.00,0.00,0.00,-0.00,1.08,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.14


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,-0.00000,-0.00003,-0.00003,-0.00001,-0.00002,0.00001
3,-0.00001,1.00000,-0.00000,0.00002,0.00001,0.00002,0.00000,-0.00001
4,-0.00000,-0.00000,1.00000,-0.00004,-0.00001,0.00004,-0.00002,0.00001
5,-0.00003,0.00002,-0.00004,1.00000,0.00002,-0.00000,0.00001,-0.00002
6,-0.00003,0.00001,-0.00001,0.00002,1.00000,-0.00001,0.00002,0.00003
7,-0.00001,0.00002,0.00004,-0.00000,-0.00001,1.00000,-0.00001,-0.00001
8,-0.00002,0.00000,-0.00002,0.00001,0.00002,-0.00001,1.00000,0.00001
9,0.00001,-0.00001,0.00001,-0.00002,0.00003,-0.00001,0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.3239686572907995

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.67084861e+08 1.58811329e+07 4.98454253e+05 1.23812515e+04
 2.87131843e+02 5.45894413e+00 8.47647700e-02 3.69191756e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999609,-0.027546,-0.004759,-0.000767,-0.000121,-0.000018,-0.000003,-3.229242e-07
1,0.027381,0.999131,-0.031045,-0.004806,-0.000755,-0.000110,-0.000016,-2.136169e-06
2,0.005577,0.030723,0.998949,-0.033151,-0.005054,-0.000735,-0.000106,-1.388023e-05
3,0.001075,0.005760,0.032748,0.998613,-0.040394,-0.005724,-0.000819,-1.087918e-04
4,0.000212,0.001130,0.006298,0.039861,0.997916,-0.049850,-0.006937,-9.155645e-04
5,0.000041,0.000219,0.001217,0.007587,0.048930,0.996061,-0.072953,-9.358244e-03
6,0.000009,0.000046,0.000259,0.001606,0.010215,0.070656,0.988659,-1.321145e-01
7,0.000002,0.000012,0.000069,0.000432,0.002741,0.018775,0.131082,9.911899e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00039109613012788813,
 0.0008689802136797597,
 0.0010505996333960432,
 0.0013868985822772517,
 0.002084091216526751,
 0.003939284367930784,
 0.011341037408263865,
 0.008810145579612239]